## This is a PoC for building the LLM enabled Chatbot with History/ Open Source

Here I am using LLM with groq based Chat app with history

Note: this will give chat option from LLM, but not from any external enabled sources or it supports any actions.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")


Get the Gemma from ChatGroq

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E5C9590FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E5C9591A60>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

Compose a Human Message and Invoke it to the Model

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager")])

AIMessage(content="Hi Ravi, it's nice to meet you!\n\nAs a Cloud & ML Technology Manager, I imagine you're involved in some exciting projects. What kind of work are you doing these days? 😄  \n\nI'm always eager to learn more about how people are using AI and cloud technologies.\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 26, 'total_tokens': 91, 'completion_time': 0.132628641, 'prompt_time': 0.004715613, 'queue_time': None, 'total_time': 0.137344254}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a3160ac-c914-46a8-b66a-7a38c1a05a1a-0', usage_metadata={'input_tokens': 26, 'output_tokens': 65, 'total_tokens': 91})

Check if it remembers what it responded using the AI Message

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager"),
        AIMessage(content="Hi Ravi, it's nice to meet you! As a Cloud & ML Technology Manager, I imagine you're involved in some exciting projects. What kind of work are you doing these days? I'm always eager to learn more about how people are using AI and cloud technologies.\n"),
        HumanMessage(content="Hi, what is My Name and my profession?"),
    ]
)

AIMessage(content="According to our conversation, your name is Ravi and you are a Cloud & ML Technology Manager. \n\nIs there anything else you'd like to know or discuss? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 105, 'total_tokens': 144, 'completion_time': 0.078561849, 'prompt_time': 0.006729572, 'queue_time': None, 'total_time': 0.085291421}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a07f718c-f620-4dcf-8faa-1354c792231b-0', usage_metadata={'input_tokens': 105, 'output_tokens': 39, 'total_tokens': 144})

##Message History Background

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_hist(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_hist=RunnableWithMessageHistory(model,get_session_history=get_session_hist)

In [7]:
config={"configurable":{"session_id":"chat1"}} # Create a session configurable

Invoke and check if the session is saved

In [11]:
response=with_message_hist.invoke(
    [HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager")],
    config=config
)

In [12]:
response.content

"Hi Ravi, it's great to meet you! \n\nBeing a Cloud & ML Technology Manager sounds like a fascinating role.  What are some of the most exciting projects you're working on right now? \n\nI'm always interested in learning about new developments in the cloud and machine learning space.  \n\nPerhaps you could tell me about:\n\n* **The types of cloud platforms you utilize (AWS, Azure, GCP, etc.)**\n* **Any specific ML models or algorithms you're currently working with**\n* **The biggest challenges you face in your role**\n\nI'm here to listen and learn! 😊  \n\n"

all the history is in Session, even if we change the config

In [17]:

config1={"configurable":{"session_id":"chat1"}}
response=with_message_hist.invoke(
    [HumanMessage(content="Hi, what is my Name?")],
    config=config1
)
response.content

'Your name is Ravi!  \n\nYou told me at the start of our conversation. 😊  \n\nIs there anything else I can help you with today, Ravi?  \n\n\n'